In [15]:
import numpy as np
from scipy.optimize import linprog

In [16]:
def dot(A, B, index):
    C = np.zeros(A.shape)
    for i in range(len(A)):
        for j in range(len(A)):
            C[i][j] += A[i][index] * B[index][j]
            if i != index:
                C[i][j] += B[i][j]

    return C

In [17]:
def find_inv(A_inv: np.array, x: np.array, index: int):
    # 1:
    l = A_inv @ x
    if l[index] == 0:
        return False

    # 2:
    l_wave = np.copy(l)
    l_wave[index] = -1.

    # 3:
    l_hat = -1. / l[index] * l_wave

    # 4:
    Q = np.identity(len(x))
    Q[:, index] = l_hat

    # 5:
    return dot(Q, A_inv, index)

In [18]:
def solve(c: np.array, A: np.array, x: np.array):
    i = 0
    while True:
        if i == 0:
            B = np.nonzero(x)[0]
            
        #1
        A_B = A[:, B]
        if i == 0:
            A_B_inv = np.linalg.inv(A_B)
        else:
            A_B_inv = find_inv(A_B_inv, A[:, j_0], k)
            
        #2
        c_B = c[B]

        #3
        u = c_B @ A_B_inv

        #4
        delta = u @ A - c

        #5 
        if (delta >= 0).all():
            return x
            
        #6
        j_0 = np.argmax(delta < 0)

        #7
        z = A_B_inv @ A[:, j_0]

        #8
        theta = np.empty(len(z))
        for i in range(len(z)):
            if z[i] > 0:
                theta[i] = x[B[i]] / z[i]
            else:
                theta[i] = np.Inf

        #9
        theta_0 = np.min(theta)

        #10
        if theta_0 == np.inf:
            raise ValueError('Целевой функционал задачи не ограничен сверху на множестве допустимых планов')
        
        #11
        k = np.argmin(theta)
        j_asterisk = B[k]

        #12
        B[k] = j_0

        #13
        x[j_0] = theta_0
        for i in range(len(B)):
            if i == k:
                continue
            x[B[i]] -= theta_0 * z[i]
        x[j_asterisk] = 0

        i += 1

In [19]:
tests = [
    {
        'c': np.array([1, 1, 0, 0, 0]),
        'A': np.array([
                [-1, 1, 1, 0, 0],
                [1, 0, 0, 1, 0],
                [0, 1, 0, 0, 1]]),
        'x': np.array([0, 0, 1, 3, 2]),
        'b': np.array([1, 3, 2])
    },
]

In [20]:
for i in range(len(tests)):
    test = tests[i]
    result1 = linprog(-test['c'], A_eq=test['A'], b_eq=test['b'])
    result2 = solve(test['c'], test['A'], test['x'])
    
    if type(result2) is str:
        assert result1.message == 'The algorithm terminated successfully and determined that the problem is infeasible.'
    else:
        assert np.allclose(result1.x, result2)